# Form field types
ipywidgets-jsonschema provides a way to dynamically generate ipywidgets from JSON Schema definitions. This guide covers how different data types are handled and how to use create widgets using JSON schema and pydantic.

## Basic Data Types

The most common types like str, int, and float are rendered as simple input fields. bool is shown as a checkbox.

In [1]:
from ipywidgets_jsonschema import Form
from pydantic import BaseModel, Field

class BasicTypes(BaseModel):
    name: str
    email: str
    age: int
    height: float
    is_active: bool
    
    
form = Form(BasicTypes)
form.show()

## Default Values
Setting the default values is simple as:

In [2]:
class DefaultValues(BaseModel):
    value: int = 10
form = Form(DefaultValues)
form.show()

When using Fields, the default value has to be specified inside the Field:

In [3]:
class DefaultValuesField(BaseModel):
    value: int = Field(default = 10)
form = Form(DefaultValues)
form.show()

# Advanced features
JSON Schema constraints such as minLength, maxLength, minimum, maximum, pattern, and format are fully supported. These constraints allow you to have fine-grained control over user input, ensuring that data is validated at the widget level.

### Using Constraints with Pydantic

To define constraints in a Pydantic model, use the Field class:

In [4]:
from pydantic import Field

class ConstraintsExample(BaseModel):
    number: int = Field(minimum= 0, maximum = 10)
    text: str = Field(min_length=3, max_length=10, pattern="^[a-zA-Z]+$")
form = Form(ConstraintsExample)
form.show()

## String formats
JSON Schema allows specifying string formats to enforce validation rules on user input. The following formats are supported:

 - email
 - hostname
 - ipv4
 - ipv6
 - uri
 - uuid

To enforce these string formats, it is simple as adding a format key to your JSON schema or pydantic model:


In [5]:
class StringFormats(BaseModel):
    email: str = Field(default="default@example.com", format="email")
    hostname: str = Field(default="default.com", format="hostname")
    ip4address: str = Field(default="127.0.0.1", format="ipv4")
    ip6address: str = Field(default="::1", format="ipv6")
    uri: str = Field(default="http://example.com", format="uri")
    uuid: str = Field(default="00000000-0000-0000-0000-000000000000", format="uuid")
form = Form(StringFormats.model_json_schema())
form.show()

ipywidgets_jsonschema also features visual feedback for user input that doesn't match the specified pattern or format. When users enter invalid data, the corresponding widget provides real-time validation cues, such as highlighting the input field or displaying an error message. 

In [6]:
class Feedback(BaseModel):
    email: str = Field(format="email")
    email_wrong: str = Field(format="email")
form = Form(Feedback)
form.show()

## Datetime

Date and time fields are rendered using special widgets that let you select a date from a calendar or set the time.

In [21]:
from datetime import date, time

class Date(BaseModel):
    date: date
    time: time
form = Form(Date)
form.show()

## List

When your schema includes a List, it will automatically generate a dynamic widget list, that allows users to:
1. Add new entries
2. Reorder list entries
3. Remove individual entries

In [8]:
from typing import List

class Fruits(BaseModel):
    fruit: List[str]
form = Form(Fruits)
form.show()

When using Set in your schema or Pydantic model, the form behaves like a list input but ensures that no duplicate entries are allowed.

In [9]:
from typing import Set
class IDs(BaseModel):
    id: Set[int]
form = Form(IDs)
form.show()

## Nested schema

We fully support **nested schemas**, allowing deeply structured form with embedded classes.

In [10]:
from typing import List

class Address(BaseModel):
    street: str
    city: str
class User(BaseModel):
    name: str
    adresses: List[Address]

form = Form(User)
form.show()

## Enum

When your model includes an Enum field, the form renders a dropdown widget, allowing you to select one of the available options.

In [11]:
from enum import Enum
class ColorEnum(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class Colors(BaseModel):
    color: ColorEnum

form = Form(Colors)
form.show()

## Anyof

We support fields defined using `Union`, which correspond to JSON Schema’s `anyOf`. These allow a field to accept multiple types. The form provides a dropdown, that allows you to select a type.

In [12]:
from typing import Union


class Item(BaseModel):
    price: Union[int, float]
form = Form(Item)
form.show()

## Object / Dictionary

In [13]:
from typing import Union, Dict
class DictUnion(BaseModel):
    settings: Dict[str, str]
form = Form(DictUnion)
form.show()


VBox(layout=Layout(width='100%'))

## Recursion


Recursive models are supported by using string annotations. This allows a model to reference itself, which is useful for things like nested categories or tree structures.

In [14]:
from typing import List

class Category(BaseModel):
    name: str
    subcategories: List['Category']

form = Form(Category)
form.show()